In [15]:
import json
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

from langchain_core.messages import (
    AIMessage,
    AnyMessage,
    HumanMessage,
    RemoveMessage,
    SystemMessage,
    ToolMessage,
)

In [16]:
load_dotenv(override=True)

True

In [17]:
coder_system_prompt = """
You are the Coding Agent in a two-stage pipeline (Planner ➜ Coder).

INPUT
------
You will receive:
1. A “Coding Plan” produced by the Planner Agent.
   • It is an ordered list of numbered tasks.  
   • Each task contains: Task Name, Details, Dependencies, Output.
2. The original user request (for reference only).
3. The structure of the DataFrame to be used in the tasks. This is the output of Pandas' `df.info()` method

OBJECTIVE
---------
Write a **single, fully-runnable Python 3 script** that accomplishes *all* tasks in the Coding Plan, in order, without omission.

STRICT RULES
------------
- **Return only code** – no prose, comments outside the script, or explanations.
- The script must be PEP 8 compliant, self-contained, and ready to run.
- Allowed libraries: Python standard library, NumPy, Pandas, Matplotlib, Scikit-Learn, PyTorch.
- If a task requires plotting, save figures to files (do not display).
- Insert clear inline comments and complete docstrings for every function, class, or complex section.
- If the plan specifies an output file name (e.g., “top_10_customers.png”), save exactly that name.
- Respect all user constraints from the original request.
- **Never ignore or reorder tasks** unless an explicit dependency forces you to combine steps.
- If the plan references data that is undefined (e.g., missing column names), raise a clear
  `ValueError` in the code rather than guessing.
- If any task is impossible with the permitted libraries, stop and raise
  `NotImplementedError` inside the script, citing the task name.

IMPLEMENTATION GUIDELINES
-------------------------
- Begin with all necessary imports.
- Encapsulate each task in a well-named function whose docstring mirrors the task description.
- Provide a `main()` function that calls task-functions in the correct order and writes/prints
  the final results as specified.
- Use type hints where helpful for readability.
- Place the customary `if __name__ == "__main__": main()` guard at the end.

FAIL-SAFE
---------
If you detect that the Coding Plan itself is ambiguous or missing critical information,
raise a `ValueError` at the top of the script explaining which task needs clarification.

OUTPUT FORMAT
-------------
Return the complete Python script **and nothing else**.

"""

In [18]:
tools = (
    [
        {"type": "code_interpreter", "container": {"type": "auto"}},
        {"type": "web_search_preview"},
    ],
)

In [19]:
import io
import os
from dataframe_to_dict import parse_dataframe_info_to_dict

df = pd.read_csv(os.getenv("PROCESSED_DATA_FILE"))
buffer = io.StringIO()
df.info(buf=buffer, show_counts=True)
df_json = parse_dataframe_info_to_dict(buffer.getvalue())


In [20]:
plan = ""
with open("planner_output.json", "r") as f:
    plan = f.read()  # Save the parsed DataFrame info to a file

In [21]:
questions = []

with open(os.getenv('QUESTIONS_FILE')) as f:
    questions = [json.loads(line)['question'] for line in f.readlines()]

In [22]:
system_message = SystemMessage(
    content=coder_system_prompt,
)

df_structure = "DataFrame Structure:\n" + json.dumps(df_json, indent=2)

plan = "Plan: "
with open("planner_output.json", "r") as f:
    plan += f.read()

original_request = questions[5]  # Assuming the original request is the 5th question

human_message = HumanMessage(
    content=plan + "\n\n"
    + "Human Request:\n" + original_request + "\n\n"
    + df_structure)
    
messages = [system_message, human_message]

In [23]:
from pydantic import BaseModel, Field

class CodeResponse(BaseModel):
    code: str = Field(description="The Python code to execute the task.")


In [24]:
llm = init_chat_model("openai:gpt-4.1", temperature=0.7, max_retries=3, output_version="responses/v1")
structured_llm = llm.with_structured_output(schema=CodeResponse)

In [25]:
resp = structured_llm.invoke(messages)

In [26]:
print(resp.code)

import pandas as pd
import numpy as np
from typing import Tuple, Optional

def load_data(file_path: str) -> pd.DataFrame:
    """
    Load the dataset from a CSV file named 'data.csv' into a Pandas DataFrame.
    """
    df = pd.read_csv(file_path)
    if 'speedAvg' not in df.columns or 'soh' not in df.columns:
        raise ValueError("Missing required columns: 'speedAvg' or 'soh' in the data.")
    return df

def fill_missing_speedavg(df: pd.DataFrame) -> pd.DataFrame:
    """
    Identify missing values in the 'speedAvg' column and replace them with the mean value of the non-missing 'speedAvg' entries.
    """
    if 'speedAvg' not in df.columns:
        raise ValueError("Column 'speedAvg' not found in DataFrame.")
    mean_speed = df['speedAvg'].mean(skipna=True)
    df['speedAvg'] = df['speedAvg'].fillna(mean_speed)
    return df

def create_speed_bins(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create a new categorical column in the DataFrame named 'speed_bin' based on the fol

In [ ]:
import pandas as pd
import numpy as np
from typing import Tuple, Optional

def load_data(file_path: str) -> pd.DataFrame:
    """
    Load the dataset from a CSV file named 'data.csv' into a Pandas DataFrame.
    """
    df = pd.read_csv(file_path)
    if 'speedAvg' not in df.columns or 'soh' not in df.columns:
        raise ValueError("Missing required columns: 'speedAvg' or 'soh' in the data.")
    return df

def fill_missing_speedavg(df: pd.DataFrame) -> pd.DataFrame:
    """
    Identify missing values in the 'speedAvg' column and replace them with the mean value of the non-missing 'speedAvg' entries.
    """
    if 'speedAvg' not in df.columns:
        raise ValueError("Column 'speedAvg' not found in DataFrame.")
    mean_speed = df['speedAvg'].mean(skipna=True)
    df['speedAvg'] = df['speedAvg'].fillna(mean_speed)
    return df

def create_speed_bins(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create a new categorical column in the DataFrame named 'speed_bin' based on the following intervals of 'speedAvg':
    0-50, 50-80, 80-100, 100+ km/h. Use intervals: [0, 50), [50, 80), [80, 100), [100, inf).
    """
    if 'speedAvg' not in df.columns:
        raise ValueError("Column 'speedAvg' not found in DataFrame.")
    bins = [0, 50, 80, 100, np.inf]
    labels = ['0-50', '50-80', '80-100', '100+']
    df['speed_bin'] = pd.cut(df['speedAvg'], bins=bins, labels=labels, right=False, include_lowest=True)
    return df

def calculate_average_soh_by_bin(df: pd.DataFrame) -> pd.DataFrame:
    """
    Group the DataFrame by 'speed_bin' and calculate the average 'soh' for each bin. Round the resulting average SOH values to two decimal places.
    """
    if 'speed_bin' not in df.columns or 'soh' not in df.columns:
        raise ValueError("Columns 'speed_bin' or 'soh' not found in DataFrame.")
    soh_by_bin = df.groupby('speed_bin', observed=True)['soh'].mean().round(2).reset_index()
    soh_by_bin = soh_by_bin.dropna(subset=['speed_bin'])
    soh_by_bin = soh_by_bin.loc[soh_by_bin['soh'].notna()]
    return soh_by_bin

def find_optimal_speed_bin(soh_by_bin: pd.DataFrame) -> str:
    """
    Identify the speed bin with the highest average SOH. If there is a tie, select the first such bin in order. If no bins exist, return an empty string.
    """
    if soh_by_bin.empty or 'soh' not in soh_by_bin.columns or 'speed_bin' not in soh_by_bin.columns:
        return ''
    max_soh = soh_by_bin['soh'].max()
    bins_with_max_soh = soh_by_bin.loc[soh_by_bin['soh'] == max_soh, 'speed_bin']
    if bins_with_max_soh.empty:
        return ''
    return str(bins_with_max_soh.iloc[0])

def main() -> None:
    df = load_data('data.csv')
    df = fill_missing_speedavg(df)
    df = create_speed_bins(df)
    soh_by_bin = calculate_average_soh_by_bin(df)
    optimal_bin = find_optimal_speed_bin(soh_by_bin)
    print(optimal_bin)


main()

